### library import & data load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from google.colab import output
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [ ]:
pd.set_option('display.max_columns', None) 

In [ ]:
kospi = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/KOSPIeng.csv')
kospi.date = kospi.date.apply(lambda x: str(x).replace('/', '-'))

merged = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/user_spec.csv')
merged['insert_date'] = merged.insert_time.apply(lambda x : str(x).split(' ')[0])

kospi_df = merged.merge(kospi[['date', 'amt', 'change']], how='left', left_on='insert_date', right_on='date')
columns = ['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'amt',
       'change']
kospi_df = kospi_df[columns]

renamed_columns = ['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'kospi', 'kospi_change']

kospi_df.columns = renamed_columns
user = kospi_df

In [ ]:
user.columns

Index(['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'kospi', 'kospi_change'],
      dtype='object')

# 1.user_spec 전처리

## 1-1.birth_year을 age로 변경

In [ ]:
birth_y = user.birth_year.apply(lambda x : int(x) if not np.isnan(x) else np.nan)
# birth year: no month infromantion
insert_y = user.insert_time.apply(lambda x : int(x.split('-')[0]))
age = insert_y - birth_y + 1
age = age.apply(lambda x : int(x) if not np.isnan(x) else np.nan)

In [ ]:
user['age'] = age

## 1-2.purpose 영어 한글 합치기 + downscaling (yearly_income, desired_amount, existing_loan_amt)

In [ ]:
mapper = { '기타' : 'ETC',
        '대환대출' : 'SWITCHLOAN',
        '생활비' : 'LIVING',
        '사업자금' : 'BUSINESS',
        '주택구입' : 'BUYHOUSE',
        '전월세보증금' : 'HOUSEDEPOSIT',
        '투자' : 'INVEST',
        '자동차구입' : 'BUYCAR'
}

purpose = user.purpose.apply(lambda x : mapper[x] if x in mapper.keys() else x)
user['purpose'] = purpose

# 6   yearly_income                        | --- scaling ---
# 11  desired_amount                       | --- scaling ---
income = user.yearly_income / 1000000
desired = user.desired_amount / 1000000
existing_loan_amt = user.existing_loan_amt / 1000000

user['yearly_income'] = income
user['desired_amount'] = desired
user['existing_loan_amt'] = existing_loan_amt

user.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,kospi,kospi_change,age
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108.0,PRIVATEBUSINESS,20151101.0,기타,자가,1.0,ETC,0.0,NaN,4.0,162.0,2626.34,-44.31,38.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30.0,SWITCHLOAN,0.0,NaN,1.0,27.0,2626.34,-44.31,55.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30.0,FREELANCER,20210901.0,기타,기타가족소유,10.0,LIVING,0.0,NaN,5.0,15.0,2626.34,-44.31,26.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62.0,EARNEDINCOME,20170101.0,정규직,자가,2.0,LIVING,0.0,NaN,7.0,344.0,2626.34,-44.31,34.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5.0,LIVING,0.0,0.0,1.0,16.0,2626.34,-44.31,23.0


## 1-3.working_period 컬럼 생성

In [ ]:
enter_company_y = user.company_enter_month.apply(lambda x: int(str(x)[:4] )if not np.isnan(x) else x)
enter_company_m = user.company_enter_month.apply(lambda x: int(str(x)[4:6]) if not np.isnan(x) else x)
insert_y = user.insert_time.apply(lambda x : int(x.split('-')[0]))
insert_m = user.insert_time.apply(lambda x : int(x.split('-')[1]))
enter_m = enter_company_y*12+enter_company_m
now_m = insert_y*12+insert_m

user['working_period'] = now_m - enter_m

## 1-3. 기대출, 개인회생 결측치 개수 컬럼

In [ ]:
import numpy as np
missing_cnt = np.array(user['personal_rehabilitation_yn'].isnull()) + np.array(user['existing_loan_cnt'].isnull())
user['missing_cnt'] = missing_cnt

## 1-4.personal_rehabilitation_yn 결측치 0으로 변경하고 personal_rehabilitation_complete_yn이랑 합쳐서 0,1,2로 변경. 이상치 제거

In [ ]:
user['personal_rehabilitation_yn'].fillna(0,inplace=True)

In [ ]:
rehab = user[['personal_rehabilitation_yn','personal_rehabilitation_complete_yn']]

In [ ]:
rehab2 = rehab[(rehab.personal_rehabilitation_yn==0)]

In [ ]:
rehab2[(rehab2.personal_rehabilitation_complete_yn==1)]

,personal_rehabilitation_yn,personal_rehabilitation_complete_yn
84244,0.0,1.0
94465,0.0,1.0
139885,0.0,1.0
1319216,0.0,1.0


In [ ]:
rehab2[(rehab2.personal_rehabilitation_complete_yn==0)]

,personal_rehabilitation_yn,personal_rehabilitation_complete_yn
4,0.0,0.0
7,0.0,0.0
9,0.0,0.0
10,0.0,0.0
18,0.0,0.0
...,...,...
1394076,0.0,0.0
1394081,0.0,0.0
1394083,0.0,0.0
1394084,0.0,0.0


In [ ]:
user.drop([84244,94465,139885,1319216],axis=0,inplace=True)

In [ ]:
user.loc[user['personal_rehabilitation_yn'] == 0, 'personal_rehabilitation_complete_yn'] = np.nan

In [ ]:
def per_re(x):
  if x==0:
    return 1
  elif x==1:
    return 0
  

user['personal_rehabilitation']=user['personal_rehabilitation_complete_yn'].apply(per_re)

In [ ]:
user.loc[user['personal_rehabilitation_yn'] == 0, 'personal_rehabilitation'] = 2

In [ ]:
user['personal_rehabilitation'].unique()

array([2., 1., 0.])

In [ ]:
user.isna().sum()

application_id                               0
user_id                                      0
birth_year                               12961
gender                                   12961
insert_time                                  0
credit_score                            105113
yearly_income                               90
income_type                                 85
company_enter_month                     171759
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn    1381503
existing_loan_cnt                       198554
existing_loan_amt                       313771
kospi                                   244203
kospi_change                            244203
age                                      12961
working_period                          171759
missing_cnt  

## 1-5.기대출 결측치 0으로 변경

In [ ]:
user[(user.existing_loan_cnt==0)]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,kospi,kospi_change,age,working_period,missing_cnt,personal_rehabilitation


In [ ]:
user['existing_loan_cnt'].fillna(0,inplace=True)

In [ ]:
user.isna().sum()

application_id                               0
user_id                                      0
birth_year                               12961
gender                                   12961
insert_time                                  0
credit_score                            105113
yearly_income                               90
income_type                                 85
company_enter_month                     171759
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn    1381503
existing_loan_cnt                            0
existing_loan_amt                       313771
kospi                                   244203
kospi_change                            244203
age                                      12961
working_period                          171759
missing_cnt  

In [ ]:
existing_loan = user[(user.existing_loan_cnt)==0]

In [ ]:
existing_loan.shape

(198554, 23)

In [ ]:
existing_loan.isna().sum()

application_id                              0
user_id                                     0
birth_year                               5245
gender                                   5245
insert_time                                 0
credit_score                            77308
yearly_income                              17
income_type                                16
company_enter_month                     30228
employment_type                            16
houseown_type                              16
desired_amount                             16
purpose                                    16
personal_rehabilitation_yn                  0
personal_rehabilitation_complete_yn    196054
existing_loan_cnt                           0
existing_loan_amt                      198554
kospi                                   31700
kospi_change                            31700
age                                      5245
working_period                          30228
missing_cnt                       

다행히 existing_loan_cnt가 0인데 existing_loan_amt가 있는 이상치는 없었다.

In [ ]:
user[(user.existing_loan_amt) >= (user.existing_loan_amt) ]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,kospi,kospi_change,age,working_period,missing_cnt,personal_rehabilitation
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108.0,PRIVATEBUSINESS,20151101.0,기타,...,0.0,NaN,4.0,162.0,2626.34,-44.31,38.0,79.0,False,2.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30.0,PRIVATEBUSINESS,20070201.0,정규직,...,0.0,NaN,1.0,27.0,2626.34,-44.31,55.0,184.0,False,2.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30.0,FREELANCER,20210901.0,기타,...,0.0,NaN,5.0,15.0,2626.34,-44.31,26.0,9.0,False,2.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62.0,EARNEDINCOME,20170101.0,정규직,...,0.0,NaN,7.0,344.0,2626.34,-44.31,34.0,65.0,False,2.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36.0,EARNEDINCOME,20210901.0,정규직,...,0.0,NaN,1.0,16.0,2626.34,-44.31,23.0,9.0,False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394207,1335338,574445,1962.0,1.0,2022-03-22 16:21:41,660.0,22.0,EARNEDINCOME,202012.0,정규직,...,0.0,NaN,5.0,102.0,2710.00,23.95,61.0,15.0,True,2.0
1394208,1832827,546472,1993.0,1.0,2022-03-22 11:53:29,710.0,26.0,EARNEDINCOME,202112.0,정규직,...,0.0,NaN,2.0,7.0,2710.00,23.95,30.0,3.0,True,2.0
1394209,256365,762698,1994.0,1.0,2022-03-22 14:42:04,640.0,12.0,EARNEDINCOME,202102.0,일용직,...,0.0,NaN,6.0,76.0,2710.00,23.95,29.0,13.0,True,2.0
1394213,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.0,75.0,EARNEDINCOME,200908.0,정규직,...,0.0,NaN,8.0,200.0,2710.00,23.95,40.0,151.0,True,2.0


## 1-6.기대출 과다자 컬럼 생성

In [ ]:
user[(user.existing_loan_amt) >= (user.yearly_income)*1.5]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,kospi,kospi_change,age,working_period,missing_cnt,personal_rehabilitation
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108.0,PRIVATEBUSINESS,20151101.0,기타,...,0.0,NaN,4.0,162.0,2626.34,-44.31,38.0,79.0,False,2.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62.0,EARNEDINCOME,20170101.0,정규직,...,0.0,NaN,7.0,344.0,2626.34,-44.31,34.0,65.0,False,2.0
11,1644897,38441,1981.0,0.0,2022-06-07 11:02:22,580.0,44.0,EARNEDINCOME,20070501.0,정규직,...,0.0,NaN,7.0,81.0,2626.34,-44.31,42.0,181.0,False,2.0
14,1754728,873333,1986.0,1.0,2022-06-07 21:16:55,940.0,30.0,PRIVATEBUSINESS,20210401.0,정규직,...,0.0,NaN,3.0,46.0,2626.34,-44.31,37.0,14.0,False,2.0
17,323850,635098,1996.0,1.0,2022-06-07 10:42:45,590.0,27.0,EARNEDINCOME,20210701.0,정규직,...,0.0,NaN,2.0,52.0,2626.34,-44.31,27.0,11.0,False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394201,569756,444524,1977.0,1.0,2022-03-22 13:40:25,680.0,58.0,EARNEDINCOME,199803.0,정규직,...,0.0,NaN,7.0,97.0,2710.00,23.95,46.0,288.0,True,2.0
1394207,1335338,574445,1962.0,1.0,2022-03-22 16:21:41,660.0,22.0,EARNEDINCOME,202012.0,정규직,...,0.0,NaN,5.0,102.0,2710.00,23.95,61.0,15.0,True,2.0
1394209,256365,762698,1994.0,1.0,2022-03-22 14:42:04,640.0,12.0,EARNEDINCOME,202102.0,일용직,...,0.0,NaN,6.0,76.0,2710.00,23.95,29.0,13.0,True,2.0
1394213,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.0,75.0,EARNEDINCOME,200908.0,정규직,...,0.0,NaN,8.0,200.0,2710.00,23.95,40.0,151.0,True,2.0


In [ ]:
user.loc[user[(user.existing_loan_amt) >= (user.yearly_income)*1.5 ].index, 'excess_of_existing_loan'] = 1

In [ ]:
user['excess_of_existing_loan'].fillna(0,inplace=True)

## 1-7.insert time에서 비율 높은 시간만 1로 나타내는 컬럼 생성

In [ ]:
import datetime
user['insert_time'] =pd.to_datetime(user['insert_time'])

In [ ]:
user['time'] = user['insert_time'].dt.time

In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1394212 entries, 0 to 1394215
Data columns (total 25 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   application_id                       1394212 non-null  int64         
 1   user_id                              1394212 non-null  int64         
 2   birth_year                           1381251 non-null  float64       
 3   gender                               1381251 non-null  float64       
 4   insert_time                          1394212 non-null  datetime64[ns]
 5   credit_score                         1289099 non-null  float64       
 6   yearly_income                        1394122 non-null  float64       
 7   income_type                          1394127 non-null  object        
 8   company_enter_month                  1222453 non-null  float64       
 9   employment_type                      1394127 non-null  ob

In [ ]:
def time(x):
  if x in ('0','1','2','3','4','5','6','9'):
    return 1
  else:
    return 0

user['max_insert_ratio_time'] = user['time'].apply(time)

## 1-8.신용카드 발급 가능 여부, 중금리대출 한도우대 대상 여부, 서민 금융법 적용 대상 여부 컬럼 생성

In [ ]:
user['credit_card'] = user['credit_score'].apply(lambda x: 1 if x >= 541 else 0)
user['mid_rate'] = user['credit_score'].apply(lambda x: 1 if x <= 850 else 0)
user['seomin_fin'] = user['credit_score'].apply(lambda x: 1 if x <= 700 else 0)

In [ ]:
user.columns

Index(['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'kospi', 'kospi_change',
       'age', 'working_period', 'missing_cnt', 'personal_rehabilitation',
       'excess_of_existing_loan', 'time', 'max_insert_ratio_time',
       'credit_card', 'mid_rate', 'seomin_fin'],
      dtype='object')

In [ ]:
user.drop(columns=['birth_year','company_enter_month','personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn','time'],inplace=True)

## 1-9. day 추가

In [ ]:
user['day'] = user.insert_time.apply(lambda x: int(str(x).split('-')[2].split(' ')[0]))

In [ ]:
user['month'] = user.insert_time.apply(lambda x: int(str(x).split('-')[1].split(' ')[0]))

In [ ]:
import datetime

def day31(x) :
  date_to_compare = datetime.datetime.strptime("20220301", "%Y%m%d")
  return (x - date_to_compare).days

user['day_from_0301'] = user['insert_time'].apply(day31)

In [ ]:
user.head()

,application_id,user_id,gender,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,kospi,kospi_change,age,working_period,missing_cnt,personal_rehabilitation,excess_of_existing_loan,max_insert_ratio_time,credit_card,mid_rate,seomin_fin,day,month,day_from_0301
0,1249046,118218,1.0,2022-06-07 06:28:18,660.0,108.0,PRIVATEBUSINESS,기타,자가,1.0,ETC,4.0,162.0,2626.34,-44.31,38.0,79.0,False,2.0,1.0,0,1,1,1,7,6,98
1,954900,553686,1.0,2022-06-07 14:29:03,870.0,30.0,PRIVATEBUSINESS,정규직,기타가족소유,30.0,SWITCHLOAN,1.0,27.0,2626.34,-44.31,55.0,184.0,False,2.0,0.0,0,1,0,0,7,6,98
2,137274,59516,1.0,2022-06-07 21:40:22,710.0,30.0,FREELANCER,기타,기타가족소유,10.0,LIVING,5.0,15.0,2626.34,-44.31,26.0,9.0,False,2.0,0.0,0,1,1,0,7,6,98
3,1570936,167320,1.0,2022-06-07 09:40:27,820.0,62.0,EARNEDINCOME,정규직,자가,2.0,LIVING,7.0,344.0,2626.34,-44.31,34.0,65.0,False,2.0,1.0,0,1,1,0,7,6,98
4,967833,33400,1.0,2022-06-07 08:55:07,630.0,36.0,EARNEDINCOME,정규직,기타가족소유,5.0,LIVING,1.0,16.0,2626.34,-44.31,23.0,9.0,False,2.0,0.0,0,1,1,1,7,6,98


In [ ]:
user.to_csv('/content/drive/Shareddrives/빅콘테스트/데이터/전처리_user_spec.csv')


# 2.merge 시키고 전처리

In [ ]:
user = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/전처리_user_spec.csv')

In [ ]:
loan_result = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/loan_result.csv')

## 2-1.productid별 최대 loan limit, 평균 loan rate & test,train split

In [ ]:
loan_result = loan_result.merge(loan_result.groupby(by='product_id').agg(max)['loan_limit'], on='product_id')

In [ ]:
loan_result = loan_result.merge(loan_result.groupby(by='product_id').agg(np.mean)['loan_rate'], on='product_id')

In [ ]:
loan_result = loan_result.rename(columns={'loan_limit_x':'loan_limit',
                   'loan_limit_y':'prod_loan_limit',
                   'loan_rate_x':'loan_rate',
                   'loan_rate_y':'prod_rate_avg'})

In [ ]:
import datetime
loan_result['loanapply_insert_time'] = pd.to_datetime(loan_result['loanapply_insert_time'])

In [ ]:
loan_result['month'] = loan_result['loanapply_insert_time'].dt.month

In [ ]:
loan = loan_result[loan_result['month']!=6]

In [ ]:
loant = loan_result[loan_result['month']==6]

In [ ]:
loan.drop(columns = ['month'], inplace=True)
loant.drop(columns = ['month'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
user = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/전처리_user_spec.csv')

In [ ]:
loan.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg
105,508290,2022-05-09 11:49:25,7,191,26000000.0,13.6,0.0,100000000.0,13.390724
106,2083697,2022-05-09 14:12:08,7,191,46000000.0,12.4,0.0,100000000.0,13.390724
107,1756892,2022-05-09 10:08:54,7,191,12000000.0,12.4,0.0,100000000.0,13.390724
108,1382679,2022-05-09 10:45:23,7,191,38000000.0,12.4,0.0,100000000.0,13.390724
109,1753,2022-05-09 17:57:29,7,191,20000000.0,13.6,0.0,100000000.0,13.390724


In [ ]:
loant.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,100000000.0,13.390724
1,2155640,2022-06-07 13:31:30,7,191,23000000.0,12.4,NaN,100000000.0,13.390724
2,1798392,2022-06-07 17:41:56,7,191,8000000.0,12.4,NaN,100000000.0,13.390724
3,1138885,2022-06-07 14:51:03,7,191,8000000.0,12.4,NaN,100000000.0,13.390724
4,1870283,2022-06-07 02:05:51,7,191,62000000.0,12.4,NaN,100000000.0,13.390724


In [ ]:
f = pd.merge(left = loan , right = user, how = "left", on = ['application_id'])

In [ ]:
f.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'prod_loan_limit',
       'prod_rate_avg', 'Unnamed: 0', 'user_id', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'employment_type',
       'houseown_type', 'desired_amount', 'purpose', 'existing_loan_cnt',
       'existing_loan_amt', 'kospi', 'kospi_change', 'age', 'working_period',
       'missing_cnt', 'personal_rehabilitation', 'excess_of_existing_loan',
       'max_insert_ratio_time', 'credit_card', 'mid_rate', 'seomin_fin', 'day',
       'month', 'day_from_0301'],
      dtype='object')

## 2-3.downscaling(loan limit)

In [ ]:
f['loan_limit'] = f['loan_limit']/1000000

## 2-4. desired랑 loan limit 차이컬럼

In [ ]:
f['dlrate']= f['desired_amount']-f['loan_limit']

## 2-5. loan에는 있는데 user spec엔 없는 애들 제외

In [ ]:
f['user_id'].isna().sum()

129

In [ ]:
ff=f.dropna(axis=0,subset=['user_id'])

In [ ]:
ff.isna().sum()

application_id                   0
loanapply_insert_time            0
bank_id                          0
product_id                       0
loan_limit                    5625
loan_rate                     5625
is_applied                       0
prod_loan_limit               1893
prod_rate_avg                 1893
Unnamed: 0                       0
user_id                          0
gender                       91626
insert_time                      0
credit_score               1243811
yearly_income                    0
income_type                      0
employment_type                  0
houseown_type                    0
desired_amount                   0
purpose                          0
existing_loan_cnt                0
existing_loan_amt          3044129
kospi                      1792455
kospi_change               1792455
age                          91626
working_period              303568
missing_cnt                      0
personal_rehabilitation          0
excess_of_existing_l

## 2-6. loan limit, loan rate nan인 애들 제외 

In [ ]:
ff['loan_limit'].isna().sum()

5625

In [ ]:
ff['loan_rate'].isna().sum()

5625

In [ ]:
fff=ff.dropna(axis=0,subset=['loan_limit'])

In [ ]:
fff.isna().sum()

application_id                   0
loanapply_insert_time            0
bank_id                          0
product_id                       0
loan_limit                       0
loan_rate                        0
is_applied                       0
prod_loan_limit                  0
prod_rate_avg                    0
Unnamed: 0                       0
user_id                          0
gender                       91585
insert_time                      0
credit_score               1243296
yearly_income                    0
income_type                      0
employment_type                  0
houseown_type                    0
desired_amount                   0
purpose                          0
existing_loan_cnt                0
existing_loan_amt          3043001
kospi                      1791625
kospi_change               1791625
age                          91585
working_period              302707
missing_cnt                      0
personal_rehabilitation          0
excess_of_existing_l

In [ ]:
fff.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_train.pickle')

# 3.위와 동일하게 test 데이터 전처리

In [ ]:
loant.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,100000000.0,13.390724
1,2155640,2022-06-07 13:31:30,7,191,23000000.0,12.4,NaN,100000000.0,13.390724
2,1798392,2022-06-07 17:41:56,7,191,8000000.0,12.4,NaN,100000000.0,13.390724
3,1138885,2022-06-07 14:51:03,7,191,8000000.0,12.4,NaN,100000000.0,13.390724
4,1870283,2022-06-07 02:05:51,7,191,62000000.0,12.4,NaN,100000000.0,13.390724


In [ ]:
user = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/전처리_user_spec.csv')

application_id

loanapply_insert_time 이거는 앞에서 주요시간대 가중치 주었구 insert랑 한번빼보기

bank_id 얘랑 프로덕트 아이디는 클러스터링한걸로 대체

product_id

loan_limit 은행별 max, 글구 desired_amount에서 빼봐야함

loan_rate 은행별 mean

is_applied

In [ ]:
ft = pd.merge(left = loant , right = user, how = "left", on = ['application_id'])

In [ ]:
ft.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'prod_loan_limit',
       'prod_rate_avg', 'Unnamed: 0', 'user_id', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'employment_type',
       'houseown_type', 'desired_amount', 'purpose', 'existing_loan_cnt',
       'existing_loan_amt', 'kospi', 'kospi_change', 'age', 'working_period',
       'missing_cnt', 'personal_rehabilitation', 'excess_of_existing_loan',
       'max_insert_ratio_time', 'credit_card', 'mid_rate', 'seomin_fin', 'day',
       'month', 'day_from_0301'],
      dtype='object')

## 3-3.downscaling(loan limit)

In [ ]:
ft['loan_limit'] = ft['loan_limit']/1000000

## 3-4. desired랑 loan limit 차이컬럼

In [ ]:
ft['dlrate']= ft['desired_amount']-ft['loan_limit']

## 3-5. loan에는 있는데 user spec엔 없는 애들 제외

In [ ]:
ft['user_id'].isna().sum()

0

## 3-6. loan limit, loan rate nan인 애들 제외 

In [ ]:
ft['loan_limit'].isna().sum()

1757

In [ ]:
ft['loan_rate'].isna().sum()

1757

In [ ]:
ft.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_test_loanlimitrate결측치안없앰.pickle')

In [ ]:
fft=ft.dropna(axis=0,subset=['loan_limit'])

In [ ]:
fft.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_test_loanlimitrate결측치없앰.pickle')

In [ ]:
ffft=ft[ft['loan_rate'].isnull()]

In [ ]:
ffft.shape

(1757, 37)

In [ ]:
ffft.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_test_loanlimitrate결측치만.pickle')

# 4.Rank 컬럼 추가: train

rank column 생성

In [ ]:
#merged = pd.read_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_train.pickle')
merged = fff.copy()

In [ ]:
merged['index_original'] = merged.index

In [ ]:
merged = merged.sort_values('loanapply_insert_time')

In [ ]:
#for train

k = 0
  
not_dummies_for_merged = ['application_id', 'user_id', 'loanapply_insert_time', 'loan_limit', 'loan_rate', 'bank_id', 'product_id', 'index_original']
merged_for_rank = merged[not_dummies_for_merged]
users = merged['user_id'].unique()

#initialize
k = 0
def all_in_one(data):
  global k
  if data.days >= 7 :
    k += 1
  return k



#merged랑 인덱스
temps1 = pd.DataFrame()
for i in tqdm(range(20000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps1 = pd.concat([temps1, temp])
############################################################################################################################

#merged랑 인덱스
temps2 = pd.DataFrame()
for i in tqdm(range(20000, 40000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps2 = pd.concat([temps2, temp])
############################################################################################################################

#merged랑 인덱스
temps3 = pd.DataFrame()
for i in tqdm(range(40000,60000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps3 = pd.concat([temps3, temp])
############################################################################################################################

#merged랑 인덱스
temps4 = pd.DataFrame()
for i in tqdm(range(60000,80000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps4 = pd.concat([temps4, temp])
############################################################################################################################

#merged랑 인덱스
temps5 = pd.DataFrame()
for i in tqdm(range(80000,100000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps5 = pd.concat([temps5, temp])
############################################################################################################################

#merged랑 인덱스
temps6 = pd.DataFrame()
for i in tqdm(range(100000,120000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps6 = pd.concat([temps6, temp])
############################################################################################################################

#merged랑 인덱스
temps7 = pd.DataFrame()
for i in tqdm(range(120000,140000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps7 = pd.concat([temps7, temp])
############################################################################################################################

#merged랑 인덱스
temps8 = pd.DataFrame()
for i in tqdm(range(140000,160000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps8 = pd.concat([temps8, temp])
############################################################################################################################

#merged랑 인덱스
temps9 = pd.DataFrame()
for i in tqdm(range(160000,180000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps9 = pd.concat([temps9, temp])
############################################################################################################################

#merged랑 인덱스
temps10 = pd.DataFrame()
for i in tqdm(range(200000,220000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps10 = pd.concat([temps10, temp])
############################################################################################################################

#merged랑 인덱스
temps11 = pd.DataFrame()
for i in tqdm(range(220000,240000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps11 = pd.concat([temps11, temp])
############################################################################################################################

#merged랑 인덱스
temps12 = pd.DataFrame()
for i in tqdm(range(240000,len(users))):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps12 = pd.concat([temps12, temp])
############################################################################################################################


  
temps = pd.concat([temps1, temps2, temps3, temps4, temps5, temps6, temps7, temps8, temps9, temps10, temps11, temps12])
###############################################################
#####################################################################
#new_merged = new_merged.merge(temp, on=initial_columns, how="left")


temps.to_pickle(f'/content/drive/Shareddrives/빅콘테스트/영주전처리_345_6/rank컬럼들_오리지널인덱스포함_train')



  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
merged_with_rank = pd.merge(left=merged, right=temps[['index_original','loan_id', 'loan_limit_rank', 'loan_rate_rank']], on=['index_original'])

In [ ]:
users = list(set(users) - set(merged_with_rank['user_id'].unique()))

In [ ]:
len(set(users) - set(merged_with_rank['user_id'].unique()))

20000

In [ ]:
#for train

k = 0
  
not_dummies_for_merged = ['application_id', 'user_id', 'loanapply_insert_time', 'loan_limit', 'loan_rate', 'bank_id', 'product_id', 'index_original']
merged_for_rank = merged[not_dummies_for_merged]
users = merged['user_id'].unique()


In [ ]:

#initialize
k = 0
def all_in_one(data):
  global k
  if data.days >= 7 :
    k += 1
  return k



#merged랑 인덱스
temps1 = pd.DataFrame()
for i in tqdm(range(len(users))):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps1 = pd.concat([temps1, temp])
############################################################################################################################

  
#temps = pd.concat([temps1, temps2, temps3, temps4, temps5, temps6, temps7, temps8, temps9, temps10, temps11, temps12])
###############################################################
#####################################################################
#new_merged = new_merged.merge(temp, on=initial_columns, how="left")


temps1.to_pickle(f'/content/drive/Shareddrives/빅콘테스트/영주전처리_345_6/rank컬럼들_오리지널인덱스포함_train_missing20000')



  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
concaaat = pd.concat([temps, temps1])

In [ ]:
final = pd.merge(left=merged, right=concaaat[['index_original', 'loan_id', 'loan_limit_rank', 'loan_rate_rank']], on='index_original')

In [ ]:
final.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_train_랭크컬럼합침_최종')

In [ ]:
final.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg,user_id,...,mid_rate,seomin_fin,day,month,day_from_0301,dlrate,index_original,loan_id,loan_limit_rank,loan_rate_rank
0,566528,2022-03-01 00:11:36,13,123,20.0,19.1,0.0,61000000.0,18.645586,681184.0,...,1.0,1.0,1.0,3.0,0.0,-19.0,8471939,0,1.0,1.0
1,180433,2022-03-01 00:12:05,19,231,16.0,15.0,0.0,100000000.0,16.242150,623737.0,...,1.0,0.0,1.0,3.0,0.0,-13.0,3078388,0,2.0,3.0
2,180433,2022-03-01 00:12:05,13,262,22.0,16.6,0.0,100000000.0,15.699650,623737.0,...,1.0,0.0,1.0,3.0,0.0,-19.0,3641628,0,1.0,5.0
3,1657888,2022-03-01 00:12:06,24,263,5.0,15.9,0.0,80000000.0,14.301378,752985.0,...,1.0,1.0,1.0,3.0,0.0,-2.0,8460865,0,3.0,4.0
4,180433,2022-03-01 00:12:06,1,61,3.0,14.9,0.0,150000000.0,12.663531,623737.0,...,1.0,0.0,1.0,3.0,0.0,0.0,4605244,0,4.0,2.0


In [ ]:
final.tail()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg,user_id,...,mid_rate,seomin_fin,day,month,day_from_0301,dlrate,index_original,loan_id,loan_limit_rank,loan_rate_rank
10264365,455157,2022-05-31 23:54:28,46,227,3.0,17.5,0.0,150000000.0,11.853471,717233.0,...,1.0,1.0,31.0,5.0,91.0,47.0,6389290,0,3.0,6.0
10264366,455157,2022-05-31 23:54:28,55,159,3.0,18.9,0.0,20000000.0,19.590801,717233.0,...,1.0,1.0,31.0,5.0,91.0,47.0,9631069,0,3.0,10.0
10264367,455157,2022-05-31 23:54:29,22,221,5.0,18.4,0.0,30000000.0,16.629038,717233.0,...,1.0,1.0,31.0,5.0,91.0,45.0,5668774,0,2.0,9.0
10264368,455157,2022-05-31 23:54:37,38,16,3.0,14.5,0.0,100000000.0,13.348448,717233.0,...,1.0,1.0,31.0,5.0,91.0,47.0,8913228,0,3.0,3.0
10264369,831190,2022-05-31 23:54:38,52,187,2.0,9.5,0.0,3000000.0,7.510221,472597.0,...,1.0,0.0,31.0,5.0,91.0,38.0,2347840,0,4.0,2.0


# 5.Rank 컬럼 추가: test 

In [ ]:
#test = pd.read_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_test_loanlimitrate결측치안없앰.pickle')
test = ft.copy()

In [ ]:
test['index_original'] = test.index

In [ ]:
test = test.sort_values(by='loanapply_insert_time')

In [ ]:
test

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,prod_loan_limit,prod_rate_avg,user_id,...,excess_of_existing_loan,max_insert_ratio_time,credit_card,mid_rate,seomin_fin,day,month,day_from_0301,dlrate,index_original
1824193,506292,2022-06-01 00:12:24,59,251,11.0,6.8,NaN,20000000.0,6.730800,669202,...,0.0,0,1,1,1,1,6,92,5.0,1824193
1116900,506292,2022-06-01 00:12:24,19,231,24.0,15.6,NaN,100000000.0,16.242150,669202,...,0.0,0,1,1,1,1,6,92,-8.0,1116900
2471422,506292,2022-06-01 00:12:24,27,148,13.0,7.2,NaN,20000000.0,7.095781,669202,...,0.0,0,1,1,1,1,6,92,3.0,2471422
1079043,1255231,2022-06-01 00:12:24,29,265,19.0,11.4,NaN,100000000.0,7.938677,694012,...,0.0,0,0,0,0,1,6,92,-9.0,1079043
3255991,506292,2022-06-01 00:12:25,56,5,2.0,18.5,NaN,45000000.0,16.255103,669202,...,0.0,0,1,1,1,1,6,92,14.0,3255991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747265,634459,2022-06-30 23:54:48,10,65,30.0,14.5,NaN,70000000.0,13.800323,244440,...,0.0,0,1,1,0,30,6,121,-25.0,747265
581892,634459,2022-06-30 23:54:48,10,149,30.0,14.5,NaN,70000000.0,13.790816,244440,...,0.0,0,1,1,0,30,6,121,-25.0,581892
1121844,634459,2022-06-30 23:54:48,19,231,11.0,16.6,NaN,100000000.0,16.242150,244440,...,0.0,0,1,1,0,30,6,121,-6.0,1121844
2639790,1288711,2022-06-30 23:54:52,35,267,3.0,13.8,NaN,150000000.0,12.473860,450880,...,0.0,0,1,1,0,30,6,121,47.0,2639790


In [ ]:
#for test

k = 0
  
not_dummies_for_merged = ['application_id', 'user_id', 'loanapply_insert_time', 'loan_limit', 'loan_rate', 'bank_id', 'product_id', 'index_original']
merged_for_rank = test[not_dummies_for_merged]
users = test['user_id'].unique()


In [ ]:

#initialize
k = 0
def all_in_one(data):
  global k
  if data.days >= 7 :
    k += 1
  return k



#merged랑 인덱스
temps1 = pd.DataFrame()
for i in tqdm(range(20000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps1 = pd.concat([temps1, temp])
############################################################################################################################

#merged랑 인덱스
temps2 = pd.DataFrame()
for i in tqdm(range(20000, 40000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps2 = pd.concat([temps2, temp])
############################################################################################################################

#merged랑 인덱스
temps3 = pd.DataFrame()
for i in tqdm(range(40000,60000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps3 = pd.concat([temps3, temp])
############################################################################################################################

#merged랑 인덱스
temps4 = pd.DataFrame()
for i in tqdm(range(60000,80000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps4 = pd.concat([temps4, temp])
############################################################################################################################

#merged랑 인덱스
temps5 = pd.DataFrame()
for i in tqdm(range(80000,100000)):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps5 = pd.concat([temps5, temp])
############################################################################################################################

#merged랑 인덱스
temps6 = pd.DataFrame()
for i in tqdm(range(100000,len(users))):
# user = users[0]
  #initialize
  k = 0
  temp = merged_for_rank[merged_for_rank['user_id']==users[i]]
  temp['loan_id'] = np.nan
  # temp 테이블에서 loanapply_insert_time 행만 추출하여 임시로 pandas.DataFrame()형태로 저장
  loanapply_insert_time_for_concat = pd.DataFrame(temp['loanapply_insert_time'])
  # shift(1)로 데이터를 한 칸 ㅇㅏㄹㅐ로 이동시킴
  loanapply_insert_time_for_concat = loanapply_insert_time_for_concat.shift(1)
  # DataFrame에서 shift(1)을 진행할 경우 가장 첫 번째 관측치(로그)에서 NaN값이 발생하므로 이를 기존 값으로 채운다.
  idxx = loanapply_insert_time_for_concat.index[0]
  loanapply_insert_time_for_concat['loanapply_insert_time'].loc[idxx] = temp['loanapply_insert_time'].iloc[0]
  # timestamp_for_concat의 컬럼명을 정의해준다.
  loanapply_insert_time_for_concat.columns = ['former_loanapply_insert_time']


  loanapply_insert_time_for_concat['loanapply_insert_time_diff'] = temp['loanapply_insert_time']-loanapply_insert_time_for_concat['former_loanapply_insert_time']
  loanapply_insert_time_for_concat['loan_id'] = loanapply_insert_time_for_concat['loanapply_insert_time_diff'].apply(all_in_one)

  # temp
  temp['loan_id'] = loanapply_insert_time_for_concat['loan_id']

  groupgroup = temp.groupby(['user_id', 'loan_id'])
  temp['loan_limit_rank'] = groupgroup['loan_limit'].rank(method="dense", ascending=False)
  temp['loan_rate_rank'] = groupgroup['loan_rate'].rank(method="dense")

  #new_merged.loc[temp.index] = temp
  temps6 = pd.concat([temps6, temp])
############################################################################################################################


temps = pd.concat([temps1, temps2, temps3, temps4, temps5, temps6])
###############################################################
#####################################################################
#new_merged = new_merged.merge(temp, on=initial_columns, how="left")


temps.to_pickle(f'/content/drive/Shareddrives/빅콘테스트/영주전처리_345_6/rank컬럼들_오리지널인덱스포함_test')



  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
final_test = pd.merge(left=test, right=temps[['index_original', 'loan_id', 'loan_limit_rank', 'loan_rate_rank']], on='index_original')

In [ ]:
final_test.to_pickle('/content/drive/Shareddrives/빅콘테스트/0_최종코드/user_loan_전처리_test_랭크컬럼합침_최종')

In [ ]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257239 entries, 0 to 3257238
Data columns (total 40 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   application_id           int64         
 1   loanapply_insert_time    datetime64[ns]
 2   bank_id                  int64         
 3   product_id               int64         
 4   loan_limit               float64       
 5   loan_rate                float64       
 6   is_applied               float64       
 7   prod_loan_limit          float64       
 8   prod_rate_avg            float64       
 9   user_id                  int64         
 10  gender                   float64       
 11  insert_time              datetime64[ns]
 12  credit_score             float64       
 13  yearly_income            float64       
 14  income_type              object        
 15  employment_type          object        
 16  houseown_type            object        
 17  desired_amount           fl

# 6.원핫인코딩

In [ ]:
ff.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'Unnamed: 0', 'user_id',
       'gender', 'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'existing_loan_cnt', 'existing_loan_amt', 'age', 'generation',
       'working_period', 'missing_cnt', 'personal_rehabilitation',
       'excess_of_existing_loan', 'max_insert_ratio_time', 'credit_grade',
       'credit_card', 'seomin_fin', 'day', 'prod_loan_limit', 'prod_rate_avg',
       'k_means_cluster', 'dlrate'],
      dtype='object')

In [ ]:
ff['k_means_cluster'] = ff['k_means_cluster'].apply(str)

In [ ]:
not_dummies = [ 'user_id', 'insert_time', 'loan_limit', 'loan_rate', 'is_applied', 'credit_score', 'yearly_income','desired_amount', 'existing_loan_cnt',
               'existing_loan_amt', 'age', 'working_period','missing_cnt', 'personal_rehabilitation',
       'excess_of_existing_loan', 'max_insert_ratio_time', 'credit_grade',
       'credit_card', 'seomin_fin', 'day', 'prod_loan_limit',
       'prod_rate_avg',  'dlrate' ]

# 'kospi', 'kospi_change', : 에러떠서 잠깐 뺐어유..

dummies = pd.get_dummies(dummy_na=False, data=ff[['gender','income_type', 'employment_type', 'houseown_type', 'purpose','k_means_cluster']])

preped = pd.concat([ff[not_dummies], dummies], axis=1)

In [ ]:
preped.columns

Index(['user_id', 'insert_time', 'loan_limit', 'loan_rate', 'is_applied',
       'credit_score', 'yearly_income', 'desired_amount', 'existing_loan_cnt',
       'existing_loan_amt', 'age', 'working_period', 'missing_cnt',
       'personal_rehabilitation', 'excess_of_existing_loan',
       'max_insert_ratio_time', 'credit_grade', 'credit_card', 'seomin_fin',
       'day', 'prod_loan_limit', 'prod_rate_avg', 'dlrate', 'gender',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_OTHERINCOME',
       'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_기타', 'employment_type_일용직',
       'employment_type_정규직', 'houseown_type_기타가족소유', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_BUSINESS',
       'purpose_BUYCAR', 'purpose_BUYHOUSE', 'purpose_ETC',
       'purpose_HOUSEDEPOSIT', 'purpose_INVEST', 'purpose_LIVING',
       'purpose_SWITCHLOAN', 'k_mean

In [ ]:
# 여기까지
preped.to_csv('/content/drive/Shareddrives/빅콘테스트/데이터정리/loanres_usrspec_final.csv', index=False)

# 7.스케일링

In [ ]:
import numpy as np

def prep_nan(func, x):
    if pd.isnull(x):
        return x
    else:
        return func(x)

# log 전처리
log_columns = ['loan_limit', 'existing_loan_amt', 'prod_loan_limit', 'working_period']

for col in log_columns:
  preped[col] = preped[col].apply(lambda x: prep_nan(np.log, x+1))

In [ ]:
# scaling
scaling_columns = ['loan_limit', 'loan_rate',  'credit_score',
       'yearly_income', 'desired_amount', 'existing_loan_cnt',
       'existing_loan_amt', 'age', 'working_period', 'missing_cnt',
       'personal_rehabilitation', 'excess_of_existing_loan',
        'credit_grade',
       'day', 'prod_loan_limit', 'prod_rate_avg', 'dlrate']

from sklearn.preprocessing import MinMaxScaler
MMScaler = MinMaxScaler()
preped[scaling_columns] = pd.DataFrame(MMScaler.fit_transform(preped[scaling_columns]), columns=scaling_columns)

In [ ]:
preped.head()

,loan_limit,loan_rate,is_applied,credit_score,yearly_income,desired_amount,existing_loan_cnt,existing_loan_amt,age,working_period,...,purpose_BUYHOUSE,purpose_ETC,purpose_HOUSEDEPOSIT,purpose_INVEST,purpose_LIVING,purpose_SWITCHLOAN,k_means_cluster_0,k_means_cluster_1,k_means_cluster_2,k_means_cluster_3
0,0.601428,0.810811,1.0,0.488889,0.0032,0.0010,0.067416,0.787480,0.452055,0.643056,...,0,0,0,0,1,0,1,0,0,0
1,0.537331,0.810811,0.0,0.533333,0.0072,0.0020,0.039326,0.756293,0.356164,0.490305,...,0,0,0,0,1,0,1,0,0,0
2,0.537331,0.702703,0.0,0.711111,0.0039,0.0080,0.011236,0.707987,0.273973,0.669085,...,0,0,0,0,0,1,1,0,0,0
3,0.679608,0.810811,0.0,0.544444,0.0031,0.0003,0.039326,0.722416,0.123288,0.356630,...,0,0,0,0,1,0,1,0,0,0
4,0.679608,0.810811,0.0,0.544444,0.0031,0.0020,0.039326,0.722416,0.123288,0.356630,...,0,0,0,0,0,1,1,0,0,0


In [ ]:
preped.to_csv('/content/drive/Shareddrives/빅콘테스트/데이터/전처리_최종본.csv')